<a href="https://colab.research.google.com/github/cwattsnogueira/rating-predictor-spam-detection-review-summarizer/blob/main/15_01_review_summarization_appt5_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gradio pandas scikit-learn --quiet

In [2]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr

In [3]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [4]:
def clean_reviews(text_list):
    return [str(t).strip().replace("\n", " ") for t in text_list if isinstance(t, str) and len(t.strip()) > 0]

In [5]:
def summarize_reviews(product_name, reviews, max_length=150):
    cleaned = clean_reviews(reviews)
    joined_text = " ".join(cleaned)
    input_text = f"summarize: {joined_text}"

    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return f"**Product:** {product_name}\n\n{summary}"

In [6]:
def summarize_from_csv(product_name, file, column_name="review_text"):
    df = pd.read_csv(file.name)
    if column_name not in df.columns:
        return f"Column '{column_name}' not found in uploaded CSV."
    reviews = df[column_name].tolist()
    return summarize_reviews(product_name, reviews)

In [7]:
with gr.Blocks() as demo:
    gr.Markdown("##  Review Summarization App")
    gr.Markdown("Upload a CSV or paste reviews manually. Include the product name to personalize the summary.")

    with gr.Tab("Manual Input"):
        product_name_manual = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headphones")
        manual_input = gr.Textbox(lines=12, placeholder="Paste multiple reviews here...")
        manual_button = gr.Button("Summarize")
        manual_output = gr.Textbox(label="Summary", lines=12, max_lines=20)

    with gr.Tab("CSV Upload"):
        product_name_csv = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headphones")
        csv_input = gr.File(label="Upload CSV")
        column_name = gr.Textbox(value="review_text", label="Column name with reviews")
        csv_button = gr.Button("Summarize CSV")
        csv_output = gr.Textbox(label="Summary", lines=12, max_lines=20)

    manual_button.click(
        lambda name, text: summarize_reviews(name, text.split("\n")),
        inputs=[product_name_manual, manual_input],
        outputs=manual_output
    )

    csv_button.click(
        summarize_from_csv,
        inputs=[product_name_csv, csv_input, column_name],
        outputs=csv_output
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e618b8e671f2c5471c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
